In [1]:
import pandas as pd
import numpy as np

In [2]:
on_faults = pd.read_pickle('../data/final_faults.pkl')

In [3]:
on_faults.dtypes

RecordID                              int64
ESS_Id                                int64
EventTimeStamp               datetime64[ns]
eventDescription                     object
ecuSoftwareVersion                   object
ecuSerialNumber                      object
ecuModel                             object
ecuMake                              object
ecuSource                             int64
spn                                   int64
fmi                                   int64
active                                 bool
activeTransitionCount                 int64
EquipmentID                          object
MCTNumber                             int64
Latitude                            float64
Longitude                           float64
LocationTimeStamp            datetime64[ns]
EventDate                    datetime64[ns]
EventTime                            object
LocationDate                 datetime64[ns]
LocationTime                         object
AcceleratorPedal                

So we need all readouts of codes other than the breakdown down within a time period of 1 week before the breakdown code occured. 

WE will have to get some sort of aggregate of events occuring up to the derate and then make new binary features of weather or not they occured. Then our result is whether or not the truck derates.  

Each column of the df will be time periods:

7-6 days prior total # of non derate codes (ie not 5246)

6-5 '''

etc.

In [4]:
window_size = pd.Timedelta('1D')

In [5]:
df = on_faults.set_index('EventTimeStamp').sort_index(ascending = False)

Make dummy columns for category of spn, get_dummies function

In [6]:
df = (pd.get_dummies(data = df, columns =['spn'])
        .filter(regex='^(spn|EquipmentID)'))

In [7]:
# derate_times = df[df['spn_5246'] == 1][['EquipmentID', 'spn_5246']] 

In [8]:
# agg_dict = {col: 'sum' for col in df.columns if (col != 'EquipmentID')}

In [9]:
# agg_dict['EquipmentID'] = lambda x : x[-1]

In [10]:
rolling = (df
            .groupby(by = 'EquipmentID')
            .rolling(window = '1D')
            .sum()         

)

In [11]:
rolling['spn_5246'].sort_values(ascending=False)

EquipmentID  EventTimeStamp     
1528         2017-02-25 07:30:09    9.0
             2017-02-25 07:30:09    9.0
1535         2016-01-11 12:12:52    9.0
             2016-01-11 12:12:53    9.0
1521         2017-01-26 13:54:07    8.0
                                   ... 
1670         2018-04-16 19:09:59    0.0
             2018-04-16 19:18:29    0.0
             2018-04-16 19:44:43    0.0
             2018-04-16 19:45:22    0.0
R1764        2015-02-25 06:08:43    0.0
Name: spn_5246, Length: 523629, dtype: float64

2 column ID for nth derate

class imbalance 

In [12]:
ml_data = rolling[rolling['spn_5246'] <= 1]

In [18]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV, KFold, train_test_split, RandomizedSearchCV
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix
from sklearn.pipeline import Pipeline

from sklearn.feature_selection import VarianceThreshold
from sklearn.preprocessing import OneHotEncoder, PolynomialFeatures, StandardScaler, MinMaxScaler
from sklearn.compose import ColumnTransformer

In [14]:
X = ml_data.drop(columns='spn_5246')

y = ml_data['spn_5246']

X_train, X_test, y_train, y_test = train_test_split(X, y,test_size=.25, random_state = 321)

In [19]:
pipe = Pipeline(
    steps = [

        ('model', DecisionTreeClassifier())
      
    ]
    
)

pipe.fit(X_train, y_train)

Pipeline(steps=[('model', DecisionTreeClassifier())])

In [21]:
print(classification_report(y_test, pipe.predict(X_test)))

              precision    recall  f1-score   support

         0.0       1.00      1.00      1.00    129730
         1.0       0.81      0.60      0.69       653

    accuracy                           1.00    130383
   macro avg       0.90      0.80      0.84    130383
weighted avg       1.00      1.00      1.00    130383

